In [2]:
import json
import numpy as np
import os

In [5]:
os.getcwd()

'c:\\Users\\thais\\Desktop\\technical-screening-master'

In [9]:
with open('groundtruth.json','r') as json_file:
    ground_truth=json.load(json_file)
with open('predictions.json', 'r') as json_file:
    predictions=json.load(json_file)

In [6]:
def find_jaccard_overlap(set_1, set_2):
    """
    Find the Jaccard Overlap (IoU) of every box combination between two sets of boxes that are in boundary coordinates.

    :param set_1: set 1, a NumPy array of dimensions (n1, 4)
    :param set_2: set 2, a NumPy array of dimensions (n2, 4)
    :return: Jaccard Overlap of each of the boxes in set 1 with respect to each of the boxes in set 2, a NumPy array of dimensions (n1, n2)
    """

    # Find intersections
    intersection = find_intersection(set_1, set_2)  # (n1, n2)

    # Find areas of each box in both sets
    areas_set_1 = (set_1[:, 2] - set_1[:, 0]) * (set_1[:, 3] - set_1[:, 1])  # (n1)
    areas_set_2 = (set_2[:, 2] - set_2[:, 0]) * (set_2[:, 3] - set_2[:, 1])  # (n2)

    # Find the union
    union = areas_set_1[:, None] + areas_set_2[None, :] - intersection  # (n1, n2)
    return intersection / union  # (n1, n2)

def find_intersection(set_1, set_2):
    """
    Find the intersection of every box combination between two sets of boxes that are in boundary coordinates.

    :param set_1: set 1, a NumPy array of dimensions (n1, 4)
    :param set_2: set 2, a NumPy array of dimensions (n2, 4)
    :return: intersection of each of the boxes in set 1 with respect to each of the boxes in set 2, a NumPy array of dimensions (n1, n2)
    """

    # Compute lower and upper bounds
    lower_bounds = np.maximum(set_1[:, None, :2], set_2[None, :, :2])  # (n1, n2, 2)
    upper_bounds = np.minimum(set_1[:, None, 2:], set_2[None, :, 2:])  # (n1, n2, 2)

    # Compute intersection dimensions
    intersection_dims = np.clip(upper_bounds - lower_bounds, a_min=0, a_max=None)  # (n1, n2, 2)
    return intersection_dims[:, :, 0] * intersection_dims[:, :, 1]  # (n1, n2)


In [25]:
for image in predictions['images']:
    print(image['location'])
    for annotated_regions in image['annotated_regions']:
        print(annotated_regions['region']['xmax'])

planes/pic6.JPG
0.6387888071174946
0.3228667914646092
planes/pic5.JPG
0.5939080947286273
0.5122928678807017
0.49209522252386223
0.5082809172452641
planes/pic12.JPG
0.419753488350937
planes/pic3.png
0.8585007164952472
0.2543777265385938
0.3322688265137087
planes/pic2.png
0.24652623250021372
0.35781445586690175
0.4285724534774521
0.4787666566884573
0.711623015689597
0.6652446437459689
0.6870827518672578
0.20958685757733933
0.25525946312725234
0.3042192676559614
0.3228667914646092
planes/pic4.png
0.19583921761257436
0.1418471489796242
0.1924104725902087
0.32018187481529603
0.28419740360762924
0.2578192267982636
0.3256948638543483
0.4636147789028956
0.5353258672785383
0.6011851962427568
0.6430071438860995
0.6344939239693625
0.58117247774923
0.7655862110792716
0.3228667914646092
planes/pic13.png
0.09943054443788893
0.5552239599949558
0.5655844461419541
0.5835751010952679
0.7964982503122378
planes/pic9.JPG
0.5504025153852999
0.6666422089113374
planes/pic10.png
0.42752559531739603
0.383329162

In [26]:
def extract_bouding_boxes(predictions, ground_truth):
    bounding_boxes_pred={}
    bounding_boxes_gt={}

    for image_pred,image_gt in zip(predictions['images'], ground_truth['images']):
        bb_pred=[]
        bb_gt=[]
        for regions_pred, regions_gt in zip(image_pred['annotated_regions'], image_gt['annotated_regions']):
            x1,y1,x2,y2=regions_pred['region']['xmin'], regions_pred['region']['ymin'], regions_pred['region']['xmax'], regions_pred['region']['ymax']
            bb_pred.append((x1,y1,x2,y2))
            X1,Y1,X2,Y2=regions_gt['region']['xmin'], regions_gt['region']['ymin'], regions_gt['region']['xmax'], regions_gt['region']['ymax']
            bb_gt.append((x1,y1,x2,y2))
        img_pred_path=image_pred['location']
        bounding_boxes_pred[img_pred_path]=bb_pred
        img_gt_path=image_gt['location']
        bounding_boxes_gt[img_pred_path]=bb_gt
    return bounding_boxes_gt,bounding_boxes_pred


bounding_boxes_gt,bounding_boxes_pred=extract_bouding_boxes(predictions, ground_truth)
print(bounding_boxes_gt,bounding_boxes_pred)


        

{'planes/pic6.JPG': [(0.5250392821595175, 0.32232196303919824, 0.6387888071174946, 0.43980025774247195)], 'planes/pic5.JPG': [(0.572080794295191, 0.4126199446574326, 0.5939080947286273, 0.45340202486342085), (0.4913963183424952, 0.525932571412702, 0.5122928678807017, 0.5817360358160355), (0.46580356291793173, 0.40077596044482977, 0.49209522252386223, 0.46691333299645144), (0.47998352616316625, 0.3451363169003685, 0.5082809172452641, 0.3954395806171641)], 'planes/pic12.JPG': [(0.36501235615108596, 0.3716813453071186, 0.419753488350937, 0.49675031286375937)], 'planes/pic3.png': [(0.7294397790204998, 0.08498590181087354, 0.8585007164952472, 0.3378607694438438), (0.19376356817575632, 0.7072111422910627, 0.2543777265385938, 0.8351905865206031), (0.2840397487568454, 0.815054036132653, 0.3322688265137087, 0.9419580357946667)], 'planes/pic2.png': [(0.1811708078773012, 0.006111546399352702, 0.24652623250021372, 0.0631073255407857), (0.31966483158698805, 0.149953053599246, 0.35781445586690175, 0